In [1]:
import pandas as pd
from collections import Counter 
import re
pd.set_option('display.max_columns', None)

In [2]:
raw_data = pd.read_csv('../data/sale_data/0_sale_raw.csv')
idxs = [i for i in raw_data.index if 'vendita' in raw_data.loc[i, 'contratto'].lower()]
raw_data = raw_data.iloc[idxs,:].reset_index(drop=True)
print('Data Shape:', raw_data.shape)
raw_data.head(2)

C:\Users\faceru\AppData\Local\Temp\ipykernel_15688\2330430630.py:1: DtypeWarning: Columns (23,24,25,26,27,29,30,31,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('../data/sale_data/0_sale_raw.csv')


Data Shape: (28519, 37)


,prezzo,stanze,m2,bagni,piano,description,Riferimento e Data annuncio,contratto,tipologia,superficie,locali,totale piani edificio,disponibilità,other_characteristics,citta,quartiere,href,regione,Posti Auto,via,unità,Data di inizio lavori e di consegna prevista,Uso edificio attuale,altre caratteristiche,spese condominio,cauzione,stato,Efficienza energetica,anno di costruzione,riscaldamento,Climatizzatore,certificazione energetica,Indice prest. energetica rinnovabile,Prestazione energetica del fabbricato,spese aggiuntive,Dati catastali,Altri dati catastali
0,€ 78.000,3,67m²,1,"Piano terra, con accesso disabili","Vendesi a Valledoria in Via Alessandro Volta, ...",Daniela - 17/12/2022,Vendita,Appartamento | Intera proprietà | Classe immob...,67 m² - Vedi dettaglio,"3 (2 camere da letto, 1 altro), 1 bagno, cucin...",2 piani,Libero,"['Cancello elettrico', 'Porta blindata', 'Terr...",Valledoria,Via Alessandro Volta,https://www.immobiliare.it/annunci/99772666/,sardegna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,€ 460.000,3,75m²,2,1°,Nell’esclusivo comprensorio di Punta Aldìa in ...,698 - 31/03/2023,Vendita,Appartamento | Intera proprietà | Classe immob...,75 m² - Vedi dettaglio,"3 (2 camere da letto, 1 altro), 2 bagni, cucin...",1 piano,Libero,"['Esposizione esterna', 'Balcone', 'Terrazza',...",San Teodoro,NaN,https://www.immobiliare.it/annunci/98803972/,sardegna,1 in parcheggio/garage comune,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
(100*raw_data.isna().sum()/raw_data.shape[0]).to_frame().rename({0:'Missings'}, axis=1).sort_values('Missings', ascending=False).round(2)

,Missings
spese aggiuntive,100.00
Prestazione energetica del fabbricato,100.00
Indice prest. energetica rinnovabile,100.00
riscaldamento,100.00
Efficienza energetica,100.00
certificazione energetica,99.99
Climatizzatore,99.99
spese condominio,99.99
anno di costruzione,99.98
cauzione,99.98


Dropping the following features:
* altre caratteristiche: lists houses characteristics already captured and with less missings by other_characteristics
* cauzione: too many missings and not enough info
* Climatizzatore
* certificazione energetica
* Data di inizio lavori e di consegna prevista	
* unità
* Prestazione energetica del fabbricato	
* Uso edificio attuale	
* Indice prest. energetica rinnovabile
* Dati catastali
* Altri dati catastali	
* spese aggiuntive
* spese condominio
* anno di costruzione
* riscaldamento
* Efficienza energetica
* stato

Features that need processings:
* other_characteristics
* posti auto
* tipologia
* via

In [4]:
print('Original N Columns', raw_data.shape[1])
features_to_drop = ['altre caratteristiche', 'cauzione', 'Climatizzatore', 'certificazione energetica', 
                    'Data di inizio lavori e di consegna prevista', 'unità', 'Prestazione energetica del fabbricato', 
                    'Uso edificio attuale', 'Indice prest. energetica rinnovabile', 'Dati catastali', 'Altri dati catastali', 
                    'spese aggiuntive', 'spese condominio', 'anno di costruzione', 'riscaldamento', 'Efficienza energetica', 'stato', 
                    'contratto', 'superficie']

raw_data = raw_data[list(set(raw_data.columns).difference(set(features_to_drop)))]
print('Cleaned N Columns', raw_data.shape[1])

Original N Columns 37
Cleaned N Columns 18


### Clean Price and M2

In [5]:
print('Observations prior to Price and M2 cleaning', raw_data.shape[0])
raw_data = raw_data[(raw_data['prezzo'].str.lower()!='prezzo su richiesta') & (~raw_data['prezzo'].str.contains('-'))].reset_index(drop=True)
raw_data['prezzo']=raw_data['prezzo'].str.replace('€ ', '').str.replace('.','').str.replace('da','').str.strip().str.replace(',','').astype(float)

Observations prior to Price and M2 cleaning 28519


In [6]:
raw_data = raw_data[raw_data['m2'].notna()]
raw_data = raw_data[raw_data['m2']!='3+']
raw_data = raw_data[(~raw_data['m2'].str.contains('/')) & (~raw_data['m2'].str.contains('T'))].reset_index(drop=True)
raw_data['m2']=raw_data['m2'].str.replace('m²', '').str.replace('.','').str.replace('da','').astype(int)
raw_data.drop_duplicates(inplace=True, ignore_index=True)
print('Observations after Price and M2 cleaning', raw_data.shape[0])

Observations after Price and M2 cleaning 17721


### Check Regione

In [7]:
raw_data['regione'].value_counts()

regione
lombardia                2475
toscana                  2410
piemonte                 2377
emilia-romagna           2348
veneto                   2238
lazio                    1817
sicilia                  1779
campania                 1766
sardegna                   73
valle-d-aosta              48
basilicata                 46
liguria                    46
trentino-alto-adige        42
puglia                     40
abruzzo                    40
molise                     39
marche                     39
calabria                   36
friuli-venezia-giulia      22
umbria                     20
emilia-Romagna             20
Name: count, dtype: int64

### Clean Data

In [8]:
raw_data['date'] = [i[i.rfind('-')+1:].strip() for i in raw_data['Riferimento e Data annuncio']]
raw_data['date'] =  pd.to_datetime(raw_data['date'], dayfirst=True)
raw_data['delta_pubblicazione'] = (raw_data['date'].max() - raw_data['date']).dt.days

### Clean Stanze

In [9]:
print('Observations prior to Stanze cleaning', raw_data.shape[0])
raw_data = raw_data[raw_data['stanze'].isin(['1', '2', '3', '4', '5', '5+'])]
raw_data.loc[raw_data['stanze']=='1', 'stanze']='monolocale'
raw_data.loc[raw_data['stanze']=='2', 'stanze']='bilocale'
raw_data.loc[raw_data['stanze']=='3', 'stanze']='trilocale'
raw_data.loc[raw_data['stanze']=='4', 'stanze']='quadrilocale'
raw_data.loc[raw_data['stanze']=='5', 'stanze']='pentalocale'
raw_data.loc[raw_data['stanze']=='5+', 'stanze']='oltre pentalocale'
print('Observations after Stanze cleaning', raw_data.shape[0])

Observations prior to Stanze cleaning 17721
Observations after Stanze cleaning 17689


### Clean Citta

In [10]:
raw_data.loc[raw_data['citta'].isna(), 'citta']='missing'

### Clean bagni

In [11]:
print('Observations prior to bagni cleaning', raw_data.shape[0])
raw_data = raw_data[raw_data['bagni'].isin(['1','2','3','3+'])].reset_index(drop=True)
print('Observations prior to bagni cleaning', raw_data.shape[0])

Observations prior to bagni cleaning 17689
Observations prior to bagni cleaning 17529


### Clean piani

In [12]:
def extract_floor_numb(column):
    if type(column)!=str:
        return -1
    floor = re.findall(r'[0-9]+', column)
    return floor[0] if len(floor)>0 else -1

raw_data['totale_piani'] = raw_data['totale piani edificio'].map(extract_floor_numb)

In [13]:
def extract_piano(str):
    if 'piani' in str.lower():
        return int(str[:2].strip())
    elif 'terra' in str.lower() or 'rialzato' in str.lower() or 'ammezzato' in str.lower():
        return 0
    elif  'su più livelli' in str.lower():
        return 2
    elif str.lower()=='missing':
        return -1
    elif 'ultimo' in str.lower():
        return 5
    else:
        return int(str[0])
    

In [14]:
print('Observations prior to piano cleaning', raw_data.shape[0])
raw_data.loc[raw_data['piano'].isna(), 'piano']='missing'
raw_data['ascensore']=raw_data['piano'].str.contains('con ascensore').astype(int)
raw_data = raw_data[~raw_data['piano'].str.contains('€')]
raw_data['piano']=raw_data['piano'].apply(extract_piano)
print('Observations prior to piano cleaning', raw_data.shape[0])

Observations prior to piano cleaning 17529
Observations prior to piano cleaning 17506


### Clean Locali

In [15]:
raw_data['cucina'] = [i[i.find('cucina'):] for i in raw_data['locali']]
raw_data.loc[raw_data['cucina'].str.len()==1, 'cucina']='no info'
raw_data.loc[raw_data['cucina'].str.contains(','), 'cucina']=raw_data['cucina'].str[:-17]
raw_data['cucina'].value_counts()

cucina
cucina abitabile         10133
cucina angolo cottura     2529
cucina a vista            2428
cucina cucinotto           988
cucina semi abitabile      816
no info                    612
Name: count, dtype: int64

In [16]:
def extract_camer(s):
    if 'camer' not in s:
        return 0
    camera = s[:s.find('camer')].strip()
    if '+' in camera and '(' in camera:
        camera = camera[4:]
    elif '(' in camera:
        camera = camera[3:]
    return camera

In [17]:
raw_data['camere'] = raw_data['locali'].apply(extract_camer)
raw_data['camere'].value_counts()

camere
2      6836
3      5137
1      2592
4      1522
5       484
0       441
6       233
7        93
8        60
10       35
9        24
12       12
11        9
14        6
15        5
20        4
30        3
16        2
13        1
175       1
18        1
48        1
99        1
26        1
32        1
17        1
Name: count, dtype: int64

### Clean quartiere

In [18]:
raw_data.loc[raw_data['quartiere'].isna(), 'quartiere']='missing'

### Clean disponibilità

In [19]:
raw_data.loc[raw_data['disponibilità'].isna(), 'disponibilità'] = 'missing'
raw_data.loc[raw_data['disponibilità'].str.contains('dal'), 'disponibilità'] = 'Futuro'
raw_data['disponibilità'].value_counts()

disponibilità
Libero     14583
missing     2458
Futuro       465
Name: count, dtype: int64

### Clean tipologia

In [20]:
raw_data['number_seps_tipologia']=raw_data['tipologia'].str.count(r'\|')
raw_data['number_seps_tipologia'].value_counts()

number_seps_tipologia
2    15356
1     1750
0      400
Name: count, dtype: int64

In [21]:
raw_data_type0 = raw_data[raw_data['number_seps_tipologia']==0].copy().reset_index(drop=True)
raw_data_type0['tipologia_casa'] = raw_data_type0['tipologia']

In [22]:
raw_data_type1 = raw_data[raw_data['number_seps_tipologia']==1].copy().reset_index(drop=True)
raw_data_type1[['tipologia_casa', 'to_fix']] = raw_data_type1['tipologia'].str.split('|', expand=True)
raw_data_type1['classe_casa']='missing'
raw_data_type1.loc[raw_data_type1['to_fix'].str.lower().str.contains('classe'), 'classe_casa']=raw_data_type1.loc[raw_data_type1['to_fix'].str.lower().str.contains('classe'), 'to_fix'].str.strip()
raw_data_type1['property_type']='missing'
raw_data_type1.loc[~raw_data_type1['to_fix'].str.lower().str.contains('classe'), 'property_type']=raw_data_type1.loc[~raw_data_type1['to_fix'].str.lower().str.contains('classe'), 'to_fix'].str.strip()
raw_data_type1.drop('to_fix', axis=1, inplace=True)

In [23]:
raw_data_type2 = raw_data[raw_data['number_seps_tipologia']==2].copy().reset_index(drop=True)
raw_data_type2[['tipologia_casa', 'tipologia_proprietà', 'classe_casa']] = raw_data_type2['tipologia'].str.split('|', expand=True)

In [24]:
raw_data = pd.concat([raw_data_type0, raw_data_type1, raw_data_type2]).reset_index(drop=True)
raw_data['tipologia_casa']=raw_data['tipologia_casa'].str.strip()
raw_data['tipologia_proprietà']=raw_data['tipologia_proprietà'].str.strip()
raw_data['classe_casa']=raw_data['classe_casa'].str.strip()
raw_data.drop('tipologia', axis=1, inplace=True)
raw_data.loc[raw_data['tipologia_casa'].isna(), 'tipologia_casa']='missing'
raw_data.loc[raw_data['classe_casa'].isna(), 'classe_casa']='missing'
raw_data.loc[raw_data['tipologia_proprietà'].isna(), 'tipologia_proprietà']='missing'
raw_data.shape

(17506, 28)

### Clean Posti auto

In [25]:
def extract_posti_auto(s):
    return sum([int(i) for i in re.findall(r'[0-9]+',s)])

In [26]:
raw_data['Posti Auto'] = raw_data['Posti Auto'].fillna('0')
raw_data['n posti auto'] = raw_data['Posti Auto'].map(extract_posti_auto)
raw_data['n posti auto'].value_counts()[:10]

n posti auto
0     7405
1     4729
2     3129
3      844
4      670
5      271
6      172
7       76
10      54
8       43
Name: count, dtype: int64

In [27]:
raw_data['box privato'] = raw_data['Posti Auto'].str.contains('privato')
raw_data['box privato'].value_counts()

box privato
False    10556
True      6950
Name: count, dtype: int64

### Clean via

In [28]:
raw_data.loc[raw_data['via'].isna(), 'via'] = 'missing'

### Other Characteristics

In [29]:
other_characteristics_full = raw_data['other_characteristics'].fillna('missing').to_list()
other_characteristics_partial = [i[1:-1].replace("'","").split(', ') for i in other_characteristics_full]
other_characteristics_clean = [i for x in other_characteristics_partial for i in x]

In [30]:
top_characteristics = Counter(other_characteristics_clean)

relevant_characteristics = {k:v for k, v in top_characteristics.items() if (v>1700 or 'infissi' in k.lower() or 'giardino' in k.lower() or k.lower() in ['idromassaggio', 'piscina'] or 'portiere' in k.lower()) 
                            and k.lower() not in ['issin', 'esposizione esterna']}
relevant_characteristics

{'Balcone': 10904,
 'Impianto tv singolo': 5404,
 'Infissi esterni in vetro / legno': 3056,
 'Porta blindata': 11845,
 'Parzialmente Arredato': 2216,
 'Cantina': 7793,
 'Esposizione doppia': 6775,
 'Arredato': 1972,
 'Caminetto': 2779,
 'Giardino privato': 5066,
 'Infissi esterni in doppio vetro / legno': 4291,
 'Terrazza': 7642,
 'Impianto di allarme': 4286,
 'Portiere intera giornata': 2075,
 'Giardino comune': 3282,
 'VideoCitofono': 5869,
 'Portiere mezza giornata': 424,
 'Cancello elettrico': 7488,
 'Fibra ottica': 7627,
 'Armadio a muro': 4354,
 'Piscina': 770,
 'Impianto tv centralizzato': 7369,
 'Infissi esterni in doppio vetro / PVC': 3428,
 'Idromassaggio': 926,
 'Mansarda': 1905,
 'Infissi esterni in triplo vetro / PVC': 576,
 'Infissi esterni in vetro / PVC': 687,
 'Infissi esterni in vetro / metallo': 729,
 'Infissi esterni in doppio vetro / metallo': 1366,
 'Giardino privato e comune': 64,
 'Infissi esterni in triplo vetro / legno': 266,
 'Infissi esterni in triplo vetro 

In [31]:
original_features = set(raw_data.columns)
raw_data['other_characteristics'] = raw_data['other_characteristics'].fillna('missing')

first_additions = [k for k in relevant_characteristics.keys() if 'infissi' not in k.lower() and 'giardino' not in k.lower() 
                   and 'portiere' not in k.lower() and 'idromassaggio' not in k.lower() and 'piscina' not in k.lower()]
for k in first_additions:
    raw_data[k.lower()] = raw_data['other_characteristics'].str.lower().str.contains(k.lower())
    
raw_data['giardino']='missing' 
raw_data['infissi']='missing' 
raw_data['portiere']=False
raw_data['piscina_idromassaggio']=False
for row in range(raw_data.shape[0]):
    if 'giardino comune' in raw_data.loc[row, 'other_characteristics'].lower():
        raw_data.loc[row, 'giardino'] = 'comune'
    if 'giardino privato' in raw_data.loc[row, 'other_characteristics'].lower() or  'Giardino privato e comune' in raw_data.loc[row, 'other_characteristics'].lower():
        raw_data.loc[row, 'giardino'] = 'privato'
        
    if 'infissi esterni in doppio' in raw_data.loc[row, 'other_characteristics'].lower():
        raw_data.loc[row, 'infissi'] = 'doppio'
    if 'infissi esterni in vetro' in raw_data.loc[row, 'other_characteristics'].lower():
        raw_data.loc[row, 'infissi'] = 'singolo'
    if 'infissi esterni in triplo' in raw_data.loc[row, 'other_characteristics'].lower():
        raw_data.loc[row, 'infissi'] = 'triplo'
        
    if 'portiere' in raw_data.loc[row, 'other_characteristics'].lower():
        raw_data.loc[row, 'portiere'] = True
    if 'piscina' in raw_data.loc[row, 'other_characteristics'].lower() or  'idromassaggio' in raw_data.loc[row, 'other_characteristics'].lower():
        raw_data.loc[row, 'piscina_idromassaggio'] = True

### Remove duplicates

In [32]:
print('N Duplicates dropped:', raw_data.shape[0]- raw_data.drop_duplicates(['prezzo','description']).shape[0])
raw_data = raw_data.sort_values('delta_pubblicazione').drop_duplicates(['prezzo','description'], ignore_index=True, keep='first')

N Duplicates dropped: 1273


### Save Data

In [33]:
useful_cols = ['citta', 'bagni', 'stanze', 'piano', 'regione',
       'm2', 'quartiere','description', 'via', 'disponibilità', 'prezzo', 
       'delta_pubblicazione', 'totale_piani',
       'ascensore', 'cucina', 'camere',
       'tipologia_casa', 'classe_casa', 'tipologia_proprietà',
       'n posti auto', 'box privato', 'balcone', 'impianto tv singolo',
       'porta blindata', 'parzialmente arredato', 'cantina',
       'esposizione doppia', 'arredato', 'caminetto', 'terrazza',
       'impianto di allarme', 'portiere','piscina_idromassaggio', 'videocitofono',
       'cancello elettrico', 'fibra ottica', 'armadio a muro', 
       'impianto tv centralizzato', 'mansarda', 'giardino',
       'infissi']

In [34]:
raw_data = raw_data[useful_cols]
(100*raw_data.isna().sum()/raw_data.shape[0]).to_frame().rename({0:'Missings'}, axis=1).sort_values('Missings', ascending=False).round(2)

,Missings
citta,0.0
balcone,0.0
porta blindata,0.0
parzialmente arredato,0.0
cantina,0.0
esposizione doppia,0.0
arredato,0.0
caminetto,0.0
terrazza,0.0
impianto di allarme,0.0


In [35]:
raw_data.to_csv('../data/sale_data/1_sale_clean.csv', index=False)